# Week 1 Assignment: Text Preprocessing

Overview:

This assignment requires applying the concepts of tokenization, stop word removal, and text normalization.

Students will write their own code for the tasks, analyze the results, and respond to exercises based on results.

Submission Requirements   
	•	Code: Submit a single Jupyter Notebook containing all the Python code.    
	•	Analysis: Include written responses to all analytical prompts in markdown cells within the notebook.    
	•	Visualization: Include all required plots and charts in the notebook.   
	•	Filename: Name the file as
			Week1_TextPreprocessing_<YourName>.pdf



In [ ]:
import pandas as pd

# URL for the SMS Spam Dataset
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/sms.tsv"

# Load the dataset
sms_data = pd.read_csv(url, sep='\t', header=None, names=['label', 'message'])

# Display the first few rows
print("First 5 rows of the SMS Spam Dataset:")
print(sms_data.head())

# Check dataset size
print("\nDataset Size:", sms_data.shape)

# Split dataset into spam and ham categories for exploration
print("\nLabel Distribution:")
print(sms_data['label'].value_counts())

## Part 1: Tokenization (20 points)


	1.	Task:   
	•	Load the SMS Spam dataset and select the first 10 messages for tokenization.  
	•	Tokensize using
			- NLTK
			- SpaCy
			- Cutom Regex based tokenizer
	•	Write a custom tokenization function using regular expressions to handle special cases like contractions and hyphenated words.

	2.	Deliverable (15 points):
	•	Python script that demonstrates tokenization using all three approaches.
	•	A brief written response comparing the outputs and discussing differences (e.g., handling contractions, special characters, and hyphenated words).

	3.	Question (5 points)
	•	How do the tokenization results differ between NLTK, SpaCy, and Regex? Which method would you prefer for preprocessing SMS messages, and why?

## Part 2: Stop Word Removal (20 points)


	1.	Task:
	•	Use NLTK & SpaCy to remove default stop words from the tokenized messages.
	•	Add custom stop words relevant to SMS messages (e.g., “free”, “click”, “call”).

	2.	Deliverable (15 points):
	•	Python script for stop word removal using NLTK 7 SpCy
	•	A bar chart comparing word frequency distributions before and after stop word removal.

	3.	Question (5 points):
	•	How does removing stop words impact the vocabulary size and word frequency distribution in the SMS dataset?

## Part 3: Text Normalization (30 points)


	1.	Task:
	•	Normalize the SMS messages by:
	•	Lowercasing all text
	•	Removing punctuation and numbers.
	•	Replacing abbreviations (e.g., “u” → “you”, “r” → “are”).
	•	Implement a reusable normalization function that combines these steps.

	2.	Deliverable (25 points):
	•	Python code for the normalization function.
	•	Example output showing the text before and after normalization.

	3.	Question (5 points):
	•	Which normalization steps had the most significant impact on SMS messages? Explain why.

## Part 4: Combining All Preprocessing Steps (30 points)

	1.	Task:
	•	Combine tokenization, stop word removal, and normalization into a single preprocessing pipeline.
	•	Apply the pipeline to first 50 messages in the SMS Spam dataset
	•	Visualize the most frequent words after preprocessing using a bar chart

	2.	Deliverable (25 points):
	•	Python code for the preprocessing pipeline.
	•	A bar chart idplaying the top 10 most frequent words.

	3.	Question (5 points):
	•	How does preprocessing improve the dataset for text classification tasks (e.g. spam detection)?